In [1]:
import gradio as gr
from hf_samdino import process_video

In [2]:
def gradio_process_video(video_file, detector, box_threshold_1, box_threshold_2, 
                         text_threshold_1, text_threshold_2, 
                         confidence, iou_threshold, 
                         frame_len, frame_stride, gif_duration):
    box_thresholds = [box_threshold_1, box_threshold_2]
    text_thresholds = [text_threshold_1, text_threshold_2]
    return process_video(video_path=video_file,
                        detector=detector,
                        dino_box_thresholds=box_thresholds,
                        dino_text_thresholds=text_thresholds,
                        yolo_confidence=confidence,
                        yolo_iou_threshold=iou_threshold,
                        frame_len=frame_len,
                        frame_stride=frame_stride,
                        gif_duration=gif_duration)

In [ ]:
iface = gr.Interface(
    fn=gradio_process_video,
    inputs=[
        gr.Video(label="Upload Video"),
        gr.Radio(choices=["Grounding DINO", "YOLO"], label="Choose a detector"),
        gr.Slider(0, 1, value=0.35, step=0.05, label="Box Threshold (DINO): Player/Referee"),
        gr.Slider(0, 1, value=0.35, step=0.05, label="Box Threshold (DINO): Basketball"),
        gr.Slider(0, 1, value=0.35, step=0.05, label="Text Threshold (DINO): Player/Referee"),
        gr.Slider(0, 1, value=0.35, step=0.05, label="Text Threshold (DINO): Basketball"),
        gr.Slider(0, 1, value=0.4, step=0.05, label="Confidence (YOLO)"),
        gr.Slider(0, 1, value=0.8, step=0.05, label="IOU Threshold (YOLO)"),
        gr.Slider(1, 500, value=100, step=1, label="Frame Length"),
        gr.Slider(1, 50, value=3, step=1, label="Frame Stride"),
        gr.Slider(1, 500, value=100, step=1, label="GIF Duration"),
    ],
    outputs=gr.Image(type="filepath"),
    title="Segment Video",
    description="Upload a video and get back a segmented GIF.",
)

iface.launch(share=True)


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://07c73e09fbeb4a4694.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Extracted 648 frames to 'output_video_frames'.
using device: cuda
Object detection of starting frame using YOLO
API Key Loaded: gw1XihSRxs3uvHsGIKUc


Counter({'player': 5, 'referee': 1})
Counter({'player': 4, 'referee': 1})
Counter({'player': 6, 'referee': 1})
Counter({'player': 6, 'referee': 1})
Counter({'player': 6, 'referee': 1})
Counter({'player': 5, 'referee': 1})
Counter({'player': 2, 'referee': 1})
Counter({'player': 4, 'referee': 1})
Counter({'player': 4, 'referee': 1})
Counter({'player': 3, 'referee': 1})
Counter({'player': 3, 'referee': 1})
Counter({'player': 4, 'referee': 1})
Counter({'player': 3, 'referee': 1})
Counter({'player': 4, 'referee': 1})
Counter({'player': 4, 'referee': 1})
Counter({'player': 5, 'referee': 1})
Counter({'player': 4, 'referee': 1})
Counter({'player': 4, 'referee': 1})
Counter({'player': 5, 'referee': 1})
Counter({'player': 2, 'referee': 1})
Counter({'player': 4, 'referee': 1})
Counter({'player': 4, 'referee': 2})
Counter({'player': 3, 'referee': 2})
Counter({'player': 3, 'referee': 2})
Counter({'player': 4, 'referee': 1})
Counter({'player': 4, 'referee': 1})
Counter({'player': 5, 'referee': 1})
C

Loading SAM2
Initializing SAM2


propagate in video:   0%|          | 0/100 [00:00<?, ?it/s]UserWarning: Memory efficient kernel not used because: (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:773.)
Falling back to all available kernels for scaled_dot_product_attention (which may have a slower speed).
propagate in video: 100%|██████████| 100/100 [00:49<00:00,  2.03it/s]


...

Creating video and gif
Video saved to output\output.mp4
Extracted 648 frames to 'output_video_frames'.
using device: cuda
Object detection of starting frame using Grounding DINO
['player. referee.', 'basketball.']
Counter({'player': 10, 'referee': 1})
Counter({'player': 14, 'referee': 1})
Counter({'player': 23, 'referee': 1})
Counter({'player': 15, 'referee': 2, 'basketball': 1})
Loading SAM2
Initializing SAM2


propagate in video: 100%|██████████| 100/100 [02:06<00:00,  1.26s/it]


Saving annotations
Saving segmented frames


100%|██████████| 100/100 [01:24<00:00,  1.18it/s]


Creating video and gif
Video saved to output\output.mp4
